In [ ]:
import os
import sys

sys.path.append(os.path.abspath('..'))

In [ ]:
from typing import Dict, List, Optional, Any
from datetime import datetime

import httpx
from loguru import logger
from smolagents import CodeAgent, LiteLLMModel, Tool, PromptTemplates
import yaml

import src.agent.adapters.tools  as tools

In [ ]:
api_base=None

# model_id="ollama/qwen3:4b"
# api_base="http://localhost:11434"

# anthorpic
# model_id="claude-3-7-sonnet-20250219"

#openai
# model_id = "gpt-4o"

# google
model_id = "gemini/gemini-2.0-flash-exp"
# model_id = "gemin/gemini-2.5-flash-preview-04-17"
# model_id = "gemini/gemini-2.5-pro-preview-05-06"

kwargs = {"base_url": "http://localhost:5050"}

In [ ]:
model = LiteLLMModel(model_id=model_id, api_base=api_base)

In [ ]:
with open("/Users/steffen/agentic_ai/src/agent/prompts/base_prompts.yaml", "r") as file:
    base_prompts = yaml.safe_load(file)


base_prompts['system_prompt'] = base_prompts['system_prompt'].replace("{{current_date}}", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
prompt_templates = PromptTemplates(**base_prompts)

In [ ]:
class FinalAnswerTool(Tool):
    name = "final_answer"
    description = "Provides a final answer to the given problem."
    inputs = {"answer": {"type": "any", "description": "The final answer to the problem"}}
    outputs = {"result": {"type": "any", "description": "The final response to the problem"}}
    output_type = "any"

    def forward(self, answer: Any) -> Any:
        return answer


In [ ]:
agent = CodeAgent(tools=[tools.CompareData(**kwargs), 
                         tools.ConvertIdToName(**kwargs),
                         tools.ConvertNameToId(**kwargs),
                         tools.GetData(**kwargs),
                         tools.GetInformation(**kwargs),
                         tools.GetNeighbors(**kwargs),
                         tools.PlotData(**kwargs),
                         FinalAnswerTool()
                        ],  
                  model=model, 
                  stream_outputs=True,
                  additional_authorized_imports=["pandas", "numpy"],
                  prompt_templates=prompt_templates,
                  max_steps=5)

In [ ]:
result = agent.run("What is the daily maximum value of PI-P0016 in Febuary 2025?")

## Tests

In [ ]:
tasks = [
    "Can you compare PI-P0017 and PI-P0016 for the first 10 days in 2025?",
    "What assets are next to asset BA100?",
    "Can you create a plot for the adjacent sensors of asset BA101 for 1st January 2025?",
    "What is the id of TI-T0022?",
    "What is the name of asset id c831fadb-d620-4007-bdda-4593038c87f9?",
    "Can you provide me the highest value for June 2025 for TI-T0022?"
]

In [ ]:
result = agent.run(tasks[0])

In [ ]:
result = agent.run(tasks[1])

In [ ]:
result = agent.run(tasks[2])

In [ ]:
result = agent.run(tasks[3])

In [ ]:
result = agent.run(tasks[4])

In [ ]:
result = agent.run(tasks[5])